<a href="https://colab.research.google.com/github/moriahsantiago/moriahsantiago/blob/main/DSMDLP_Module3_Part1_Prediction_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge: K-Fold Cross Validation
Using this code, revise the Logistic Regression training procedure to instead apply a 10-fold cross validation. You will be using the same dataset as the previous assignment, and an abbreviate preprocessing pipeline has been provided.

For this assignment, our goal is to train a model to predict whether students are likely to complete their next math assignment.

## Dataset

This assignment will utilize student data collected from the ASSISTments digital learning platform. The dataset contains data from 18,345 unique students across 37,268 assignments. For each of these, our goal is to predict whether each student will complete the assignment using their past performance and behavior in the system. Several student level features have been aggregated for each student up to the moment that they were given the target assignment, and we want to examine whether this information can be used to predict their future performance (as measured by completion of the next assignment).

**The dataset can be downloaded from Canvas or using this direct link:
[ASSISTments Assignments Dataset](https://drive.google.com/file/d/1oEEKSkDPn8RBM6oa9eNmXYIFBmxqFQBm/view?usp=sharing)**

The dataset used in this assignment has been sampled from a larger publicly-available dataset published in the International Conference on Educational Data Mining:

[Prihar, E., Syed, M., Ostrow, K., Shaw, S., Sales, A., & Heffernan, N. (2022, July). Exploring common trends in online educational experiments. *In Proceedings of the 15th International Conference on Educational Data Mining.*](https://educationaldatamining.org/edm2022/proceedings/2022.EDM-long-papers.3/index.html)

**A description of each column and further context for the data can be found in that paper.**

#Data Loading and Preprocessing
Download the **assistments_control_assignments_with_priors.csv** file from Canvas or the link above. Run the first code cell below to upload the dataset. The second code cell below uses the pandas library to read the file into a Dataframe and displays the number of rows and columns as well as a sample of the loaded data.



In [1]:
from google.colab import files
dataset = files.upload()
filename = list(dataset.keys())[0]
print(f"{filename} has been uploaded")

Saving assistments_control_assignments_with_priors (2).csv to assistments_control_assignments_with_priors (2).csv
assistments_control_assignments_with_priors (2).csv has been uploaded


In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv(filename)

# Drop rows with missing values
df = df.dropna()

# Print the shape of the dataset
print("\nShape of the dataset (rows, columns):", df.shape)

# Display a sample of the dataset
df



Shape of the dataset (rows, columns): (37268, 34)


,problem_set_id,student_id,assignment_completed,normalized_student_learning,0_student_prior_skill_builders,0_student_prior_problem_sets,0_student_prior_attempted_problems,0_student_prior_completed_problems,student_prior_started_skill_builder_count,student_prior_skill_builder_percent_completed,...,class_prior_skill_builder_percent_completed,class_prior_started_problem_set_count,class_prior_problem_set_percent_completed,class_prior_completed_problem_count,class_prior_median_first_response_time,class_prior_median_time_on_task,class_prior_average_attempt_count,class_prior_average_correctness,teacher_account_age_in_days,opportunity_zone
0,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
1,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
2,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
3,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
4,PSAUUKY,137672,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37263,PSA25TA,1035779,1,-0.125259,0,1,0,0,8,0.875,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37264,PSA25TA,1031811,1,1.128823,0,1,0,0,10,1.000,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37265,PSA25TA,1035614,0,-1.379341,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1
37266,PSA25TA,1035616,1,1.128823,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1


##Descriptive Statistics

Following the previous assignment, the code below identifies all the features that were considered in the previous assignment in addition to a set of features for which there is low correlation. Feel free to replace these features with the set that you identified in the previous assignment.

In [5]:
# The list of all potential features we can utilize
# Note: class- and teacher-level variables are omitted for this assignment,
#       but these may be useful in other contexts
all_features = ['0_student_prior_skill_builders',
                      '0_student_prior_problem_sets',
                      '0_student_prior_attempted_problems',
                      '0_student_prior_completed_problems',
                      'student_prior_started_skill_builder_count',
                      'student_prior_skill_builder_percent_completed',
                      'student_prior_started_problem_set_count',
                      'student_prior_problem_set_percent_completed',
                      'student_prior_completed_problem_count',
                      'student_prior_median_first_response_time',
                      'student_prior_median_time_on_task',
                      'student_prior_average_attempt_count',
                      'student_prior_average_correctness',
                      'opportunity_zone']

# The following represents one possible permutation of features
low_corr_features = ['0_student_prior_completed_problems',
                      'student_prior_started_skill_builder_count',
                      'student_prior_skill_builder_percent_completed',
                      'student_prior_problem_set_percent_completed',
                      'student_prior_completed_problem_count',
                      'student_prior_median_first_response_time',
                      'student_prior_average_attempt_count',
                      'student_prior_average_correctness',
                      'opportunity_zone']

dependent_variable = "assignment_completed"
df_features = df[all_features]

# Display the means and standard deviations
stats = {}
stats['Mean'] = df_features.mean(numeric_only=True)
stats['SD'] = df_features.std(numeric_only=True)

feature_statistics = pd.DataFrame(stats)

print(f"The dataset contains a total of {len(df)} rows and {len(df.columns)} columns")
print(f"{len(all_features)} potential feature columns have been identified:")
print(feature_statistics.round(2))
print("==========")
print(f"Dependent Measure: {dependent_variable}")
print(f"{round(df[dependent_variable].mean()*100,2)}% of the {df[dependent_variable].sum()} total assignments were completed")

The dataset contains a total of 37268 rows and 34 columns
14 potential feature columns have been identified:
                                                 Mean       SD
0_student_prior_skill_builders                   0.19     0.40
0_student_prior_problem_sets                     0.24     0.43
0_student_prior_attempted_problems               0.06     0.24
0_student_prior_completed_problems               0.06     0.24
student_prior_started_skill_builder_count        6.45     8.73
student_prior_skill_builder_percent_completed    0.65     0.40
student_prior_started_problem_set_count         18.26    30.63
student_prior_problem_set_percent_completed      0.63     0.42
student_prior_completed_problem_count          175.76   268.13
student_prior_median_first_response_time        97.14  5148.59
student_prior_median_time_on_task              106.91  5148.64
student_prior_average_attempt_count              1.32     0.47
student_prior_average_correctness                0.62     0.23
opportuni

#Logistic Regression

With our set of selected features your goal is to modify the code below to instead implement a K-fold cross validation. Refer to the ASSISTments assignment for hints as to how to get started if you are unsure.

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

# Use your existing df, selected_features, and dependent_variable
X = df[selected_features]
y = df[dependent_variable]

param_grid = [
    {'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 3},
    {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 3},
    {'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 3},
    {'max_depth': 2, 'min_samples_split': 5, 'min_samples_leaf': 3},
    {'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 3},
    {'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 3}
]

print("Evaluating AUC for Decision Tree hyperparameters:\n")
for params in param_grid:
    dt = DecisionTreeClassifier(**params, random_state=42)
    auc = cross_val_score(dt, X, y, cv=10, scoring='roc_auc').mean()
    print(f"Params: {params}, AUC: {auc:.6f}")


Evaluating AUC for Decision Tree hyperparameters:

Params: {'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 3}, AUC: 0.732483
Params: {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 3}, AUC: 0.723712
Params: {'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 3}, AUC: 0.637232
Params: {'max_depth': 2, 'min_samples_split': 5, 'min_samples_leaf': 3}, AUC: 0.712818
Params: {'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 3}, AUC: 0.722427
Params: {'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 3}, AUC: 0.637232
